![The Pokémon Logo](images/pokemon-logo.png)

# **Introduction**

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

#### Importing Necessary Libraries

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Loading Data

There are two .csv's that I'll use for this project:

1. Historical Data from TCGPlayer (.csv)
2. Most Popular Pokemon Data (.csv)

In [61]:
# reading in the historical data
hist_df = pd.read_csv('tcgplayer-data.csv')

# showing the first five rows
hist_df.head()

,Card,Set,Number,Rarity,Condition,Volatility,3/18 to 3/24,3/25 to 3/31,4/1 to 4/7,4/8 to 4/14,...,1/6 to 1/12,1/13 to 1/19,1/20 to 1/26,1/27 to 2/2,2/3 to 2/9,2/10 to 2/16,2/17 to 2/23,2/24 to 3/2,3/3 to 3/9,3/10 to 3/16
0,Gengar VMAX,SWSH08: Fusion Strike,271/264,Secret Rare,Near Mint,Med Volatility,228.68,252.39,289.66,312.91,...,657.73,697.20,732.91,744.80,735.06,718.11,719.55,699.55,682.49,679.85
1,Gengar ex,SV05: Temporal Forces,193/162,Ultra Rare,Near Mint,Med Volatility,19.05,13.13,13.78,12.70,...,22.79,24.81,24.98,24.79,24.57,22.54,24.16,25.13,24.15,22.57
2,Gengar,SWSH11: Lost Origin Trainer Gallery,TG06/TG30,Ultra Rare,Near Mint,Med Volatility,2.47,2.57,2.98,2.35,...,10.22,13.40,16.17,15.35,15.00,15.59,19.98,18.98,17.25,16.61
3,Charizard,Base Set,004/102,Holo Rare,Near Mint,Med Volatility,385.66,386.97,370.05,369.79,...,442.48,427.29,406.44,441.24,440.18,378.71,398.63,449.77,453.59,462.27
4,Charizard GX,SM - Burning Shadows,150/147,Secret Rare,Near Mint,Indeterminate Volatility,397.94,397.94,413.30,413.30,...,420.70,411.27,414.50,414.50,414.50,416.27,423.25,423.90,418.99,418.99


In [62]:
# displaying the info of the historical prices data frame
hist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 58 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Card            103 non-null    object 
 1   Set             103 non-null    object 
 2   Number          103 non-null    object 
 3   Rarity          103 non-null    object 
 4   Condition       103 non-null    object 
 5   Volatility      103 non-null    object 
 6   3/18 to 3/24    103 non-null    float64
 7   3/25 to 3/31    103 non-null    float64
 8   4/1 to 4/7      103 non-null    float64
 9   4/8 to 4/14     103 non-null    float64
 10  4/15 to 4/21    103 non-null    float64
 11  4/22 to 4/28    103 non-null    float64
 12  4/29 to 5/5     103 non-null    float64
 13  5/6 to 5/12     103 non-null    float64
 14  5/13 to 5/19    103 non-null    float64
 15  5/20 to 5/26    103 non-null    float64
 16  5/27 to 6/2     103 non-null    float64
 17  6/3 to 6/9      103 non-null    flo

In [63]:
# reading in the popularity data
pop_df = pd.read_csv('pokemon-favorites-list.csv')

# showing the first five rows
pop_df.head()

,Pokemon,Number of votes,Rank
0,Charizard,1107,1
1,Gengar,1056,2
2,Arcanine,923,3
3,Bulbasaur,710,4
4,Blaziken,613,5


#### Upkeep, Merging, and Cleaning Data

Upkeep:
*   Taking a look at both dateframes and getting rid of any columns that aren't going to factor into our analysis.
*   Rearranging columns if they need to be so that readability won't be an issue.
*   Converting title names for consistency
*   Renaming the 'Card' column to 'Pokemon'

In [64]:
print(hist_df.columns)  # remove 'Volatility'

Index(['Card', 'Set', 'Number', 'Rarity', 'Condition', 'Volatility',
       '3/18 to 3/24', '3/25 to 3/31', '4/1 to 4/7', '4/8 to 4/14',
       '4/15 to 4/21', '4/22 to 4/28', '4/29 to 5/5', '5/6 to 5/12',
       '5/13 to 5/19', '5/20 to 5/26', '5/27 to 6/2', '6/3 to 6/9',
       '6/10 to 6/16', '6/17 to 6/23', '6/24 to 6/30', '7/1 to 7/7',
       '7/8 to 7/14', '7/15 to 7/21', '7/22 to 7/28', '7/29 to 8/4',
       '8/5 to 8/11', '8/12 to 8/18', '8/19 to 8/25', '8/26 to 9/1',
       '9/2 to 9/8', '9/9 to 9/15', '9/16 to 9/22', '9/23 to 9/29',
       '9/30 to 10/6', '10/7 to 10/13', '10/14 to 10/20', '10/21 to 10/27',
       '10/28 to 11/3', '11/4 to 11/10', '11/11 to 11/17', '11/18 to 11/24',
       '11/25 to 12/1', '12/2 to 12/8', '12/9 to 12/15', '12/16 to 12/22',
       '12/23 to 12/29', '12/30 to 1/5', '1/6 to 1/12', '1/13 to 1/19',
       '1/20 to 1/26', '1/27 to 2/2', '2/3 to 2/9', '2/10 to 2/16',
       '2/17 to 2/23', '2/24 to 3/2', '3/3 to 3/9', '3/10 to 3/16'],
      dtype='o

In [65]:
print(pop_df.columns)   # remove 'Number of votes'

Index(['Pokemon', 'Number of votes', 'Rank'], dtype='object')


#### Card Data and Historical Prices

General upkeep and renaming to prepare for a more seamless merger.  'Volatility' was a novel idea, but it seems like half of the cards have 'Indeterminate Volatility' and that doesn't help with our analysis in this case.

In [66]:
hist_clean = hist_df.drop(columns = ['Volatility'])    # removing the 'Volatility' column

# checking to see if the change was successful
hist_clean.head()  

,Card,Set,Number,Rarity,Condition,3/18 to 3/24,3/25 to 3/31,4/1 to 4/7,4/8 to 4/14,4/15 to 4/21,...,1/6 to 1/12,1/13 to 1/19,1/20 to 1/26,1/27 to 2/2,2/3 to 2/9,2/10 to 2/16,2/17 to 2/23,2/24 to 3/2,3/3 to 3/9,3/10 to 3/16
0,Gengar VMAX,SWSH08: Fusion Strike,271/264,Secret Rare,Near Mint,228.68,252.39,289.66,312.91,314.11,...,657.73,697.20,732.91,744.80,735.06,718.11,719.55,699.55,682.49,679.85
1,Gengar ex,SV05: Temporal Forces,193/162,Ultra Rare,Near Mint,19.05,13.13,13.78,12.70,11.52,...,22.79,24.81,24.98,24.79,24.57,22.54,24.16,25.13,24.15,22.57
2,Gengar,SWSH11: Lost Origin Trainer Gallery,TG06/TG30,Ultra Rare,Near Mint,2.47,2.57,2.98,2.35,2.60,...,10.22,13.40,16.17,15.35,15.00,15.59,19.98,18.98,17.25,16.61
3,Charizard,Base Set,004/102,Holo Rare,Near Mint,385.66,386.97,370.05,369.79,360.99,...,442.48,427.29,406.44,441.24,440.18,378.71,398.63,449.77,453.59,462.27
4,Charizard GX,SM - Burning Shadows,150/147,Secret Rare,Near Mint,397.94,397.94,413.30,413.30,408.31,...,420.70,411.27,414.50,414.50,414.50,416.27,423.25,423.90,418.99,418.99


#### Popularity Rankings

More upkeep and removals...

In [67]:
pop_clean = pop_df.drop(columns = ['Number of votes'])  # removing the 'Number of votes' column
pop_clean.rename(columns = {'Rank': 'Popularity'}, inplace = True)  # renaming 'Rank' to 'Popularity Ranking' for clarity

# checking to see if the changes were successful
pop_clean.head()

,Pokemon,Popularity
0,Charizard,1
1,Gengar,2
2,Arcanine,3
3,Bulbasaur,4
4,Blaziken,5


In [68]:
# displaying the popularity info
pop_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809 entries, 0 to 808
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pokemon     809 non-null    object
 1   Popularity  809 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.8+ KB


#### Data Consistency

Cards come from different sets, eras, and as such have different gimmicks.  Every other column, however, is distinct thanks to the web scraper and foresight.  The popular column only has a few changes that're needed.

That being said, we do have missing values for cards that were released in the last 52 weeks.  We will change those to 'NULL' to avoid price inaccuracies that affect our data as we graph it.

In [69]:
# replacing '0.00' with null values in the 'Price' column
hist_clean.replace(0.0, np.nan, inplace = True)

# checking for card 'Number' '238/191', which is a Pikachu from a set that released late last year to check if our change worked
hist_clean.loc[hist_clean['Number'] == '238/191']

,Card,Set,Number,Rarity,Condition,3/18 to 3/24,3/25 to 3/31,4/1 to 4/7,4/8 to 4/14,4/15 to 4/21,...,1/6 to 1/12,1/13 to 1/19,1/20 to 1/26,1/27 to 2/2,2/3 to 2/9,2/10 to 2/16,2/17 to 2/23,2/24 to 3/2,3/3 to 3/9,3/10 to 3/16
79,Pikachu ex,SV08: Surging Sparks,238/191,Special Illustration Rare,Near Mint,NaN,NaN,NaN,NaN,NaN,...,476.18,485.95,474.56,468.81,461.43,457.9,456.22,456.14,442.02,442.08


#### Final Preparations to Merge

The Data is still pretty dirty on the names of the cards themselves despite letters (ex, GX, etc.) being descriptive.  It makes it very hard to apply their popularity rankings, so we're going to split and explode on the 'Card' column, and rename the 'Card' in preparation for our merger.

In [70]:
# we split the text in the cells on whitespace, then explode
hist_expand = hist_clean.assign(Card = hist_clean['Card'].str.split()).explode('Card', ignore_index = True)

# renaming 'Card' to 'Pokemon'
hist_expand.rename(columns = {'Card': 'Pokemon'}, inplace = True)

# checking
hist_expand.head()

,Pokemon,Set,Number,Rarity,Condition,3/18 to 3/24,3/25 to 3/31,4/1 to 4/7,4/8 to 4/14,4/15 to 4/21,...,1/6 to 1/12,1/13 to 1/19,1/20 to 1/26,1/27 to 2/2,2/3 to 2/9,2/10 to 2/16,2/17 to 2/23,2/24 to 3/2,3/3 to 3/9,3/10 to 3/16
0,Gengar,SWSH08: Fusion Strike,271/264,Secret Rare,Near Mint,228.68,252.39,289.66,312.91,314.11,...,657.73,697.20,732.91,744.80,735.06,718.11,719.55,699.55,682.49,679.85
1,VMAX,SWSH08: Fusion Strike,271/264,Secret Rare,Near Mint,228.68,252.39,289.66,312.91,314.11,...,657.73,697.20,732.91,744.80,735.06,718.11,719.55,699.55,682.49,679.85
2,Gengar,SV05: Temporal Forces,193/162,Ultra Rare,Near Mint,19.05,13.13,13.78,12.70,11.52,...,22.79,24.81,24.98,24.79,24.57,22.54,24.16,25.13,24.15,22.57
3,ex,SV05: Temporal Forces,193/162,Ultra Rare,Near Mint,19.05,13.13,13.78,12.70,11.52,...,22.79,24.81,24.98,24.79,24.57,22.54,24.16,25.13,24.15,22.57
4,Gengar,SWSH11: Lost Origin Trainer Gallery,TG06/TG30,Ultra Rare,Near Mint,2.47,2.57,2.98,2.35,2.60,...,10.22,13.40,16.17,15.35,15.00,15.59,19.98,18.98,17.25,16.61


#### Merging Dataframes

With the newly exploded historical dataframe, we're going to merge on 'Pokemon', and then apply the rankings to the card names themselves.  Afterward, we'll drop the rows without rankings, round out the numbers so they're not of the type 'float64' anymore, and reset our index.

In [71]:
# left-joining on hist_clean and creating a new merged dataframe; I was getting an error 
final_df = hist_expand.merge(pop_clean, on = "Pokemon", how = "left")

# drop rows that have 'NaN' in the 'Popularity' column
final_df = final_df.dropna(subset = "Popularity")

# change from float to int for better presentation (1.0 to 1)
final_df["Popularity"] = final_df["Popularity"].astype("Int64")

# reset index
final_df.reset_index(drop = True, inplace = True)

# checking
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 58 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Pokemon         103 non-null    object 
 1   Set             103 non-null    object 
 2   Number          103 non-null    object 
 3   Rarity          103 non-null    object 
 4   Condition       103 non-null    object 
 5   3/18 to 3/24    99 non-null     float64
 6   3/25 to 3/31    100 non-null    float64
 7   4/1 to 4/7      100 non-null    float64
 8   4/8 to 4/14     100 non-null    float64
 9   4/15 to 4/21    100 non-null    float64
 10  4/22 to 4/28    100 non-null    float64
 11  4/29 to 5/5     101 non-null    float64
 12  5/6 to 5/12     102 non-null    float64
 13  5/13 to 5/19    102 non-null    float64
 14  5/20 to 5/26    102 non-null    float64
 15  5/27 to 6/2     102 non-null    float64
 16  6/3 to 6/9      102 non-null    float64
 17  6/10 to 6/16    102 non-null    flo

#### Price Analyses
Now, we'll take some normal financial calculations, numbers, and essentially treat these cards like stock.

In [72]:
# matching on '/' and 'to' to group columns together for easier calculations
yr_prices = [col for col in final_df.columns if '/' in col and 'to' in col]

# taking traditional price stats
final_df['Average'] = final_df[yr_prices].mean(axis = 1).round(2) # rounding to 2 decimal places
final_df['Min'] = final_df[yr_prices].min(axis = 1)
final_df['Max'] = final_df[yr_prices].max(axis = 1)
final_df['Range'] = final_df['Max'] - final_df['Min']

# checking...
final_df.head()

,Pokemon,Set,Number,Rarity,Condition,3/18 to 3/24,3/25 to 3/31,4/1 to 4/7,4/8 to 4/14,4/15 to 4/21,...,2/10 to 2/16,2/17 to 2/23,2/24 to 3/2,3/3 to 3/9,3/10 to 3/16,Popularity,Average,Min,Max,Range
0,Gengar,SWSH08: Fusion Strike,271/264,Secret Rare,Near Mint,228.68,252.39,289.66,312.91,314.11,...,718.11,719.55,699.55,682.49,679.85,2,422.19,228.68,744.80,516.12
1,Gengar,SV05: Temporal Forces,193/162,Ultra Rare,Near Mint,19.05,13.13,13.78,12.70,11.52,...,22.54,24.16,25.13,24.15,22.57,2,15.16,9.10,25.13,16.03
2,Gengar,SWSH11: Lost Origin Trainer Gallery,TG06/TG30,Ultra Rare,Near Mint,2.47,2.57,2.98,2.35,2.60,...,15.59,19.98,18.98,17.25,16.61,2,7.01,2.35,19.98,17.63
3,Charizard,Base Set,004/102,Holo Rare,Near Mint,385.66,386.97,370.05,369.79,360.99,...,378.71,398.63,449.77,453.59,462.27,1,383.34,323.85,462.27,138.42
4,Charizard,SM - Burning Shadows,150/147,Secret Rare,Near Mint,397.94,397.94,413.30,413.30,408.31,...,416.27,423.25,423.90,418.99,418.99,1,422.71,397.94,439.93,41.99


#### Advanced Stats

Having merged the popularity, we can calculate an 'advanced' stat like:

**Inverted Popularity**

Since lower values indicate a higher popularity, we'll invert the score based on the values in this table.  This is intuitive, especially for the future calculations we'll be doing, i.e. weighted prices, fluctuations, and deviations from popularity.

$$\text{Inverted\_Popularity} = \max(\text{Popularity}) - \text{Popularity} + 1$$



In [73]:
# creating the inverted popularity variable for our future calculations
max_popularity = final_df['Popularity'].max() # taking the max
final_df['Inverted Pop.'] = max_popularity - final_df['Popularity'] + 1 # formula seen above

# checking...
final_df.head()

,Pokemon,Set,Number,Rarity,Condition,3/18 to 3/24,3/25 to 3/31,4/1 to 4/7,4/8 to 4/14,4/15 to 4/21,...,2/17 to 2/23,2/24 to 3/2,3/3 to 3/9,3/10 to 3/16,Popularity,Average,Min,Max,Range,Inverted Pop.
0,Gengar,SWSH08: Fusion Strike,271/264,Secret Rare,Near Mint,228.68,252.39,289.66,312.91,314.11,...,719.55,699.55,682.49,679.85,2,422.19,228.68,744.80,516.12,360
1,Gengar,SV05: Temporal Forces,193/162,Ultra Rare,Near Mint,19.05,13.13,13.78,12.70,11.52,...,24.16,25.13,24.15,22.57,2,15.16,9.10,25.13,16.03,360
2,Gengar,SWSH11: Lost Origin Trainer Gallery,TG06/TG30,Ultra Rare,Near Mint,2.47,2.57,2.98,2.35,2.60,...,19.98,18.98,17.25,16.61,2,7.01,2.35,19.98,17.63,360
3,Charizard,Base Set,004/102,Holo Rare,Near Mint,385.66,386.97,370.05,369.79,360.99,...,398.63,449.77,453.59,462.27,1,383.34,323.85,462.27,138.42,361
4,Charizard,SM - Burning Shadows,150/147,Secret Rare,Near Mint,397.94,397.94,413.30,413.30,408.31,...,423.25,423.90,418.99,418.99,1,422.71,397.94,439.93,41.99,361


****

#### Popularity Weighting

We'll combine the average price with the Pokemon's popularity, adjusting the card's price based on the popularity in the card.

*This tells us* how a Pokemon's popularity affects its value and whether the popularity of the Pokemon in the card directly impacts pricing, or if there are other variables at work. 

****

#### Popularity-Weighted Price Fluctuation

This applies similar logic as above to the price range, showcasing how the volatility of the card's price may be affected by the popularity of the Pokemon.

*This tells us*: dependant upon the range of prices, specifically price spikes, would indicate either demand-driven fluctuations, risk, other factors like rarity are at play, and whether popularity stablizes or destablizes pricing.

****

#### Price Deviation from Popularity

This compares a card's price to the average price of all the other cards featuring that Pokemon.  If said card is so much more expensive than the other cards with that Pokemon, it's an outlier.

*This tells us* if certain cards break the norm, expected pricing pattern and helps determine - like the weighted pricing fluctuation above - if pricing is influenced by other factors.

                    *author's note*: This would especially be useful for me as I'm trying to find undervalued/overpriced cards.

In [75]:
# popularity weighting by taking average * inverted popularity
final_df['Weighted Price (Pop.)'] = final_df['Average'] * final_df['Inverted Pop.']

# pop. weight. price fluctuations
final_df['Price Fluctuations (Pop.)'] = final_df['Range'] * final_df['Inverted Pop.']

# price deviations are a little more intricate because we have to transform on the popularity by average prices
popularity_group_mean = final_df.groupby('Popularity')['Average'].transform('mean')
final_df['Price Deviation from Pop'] = final_df['Average'] - popularity_group_mean

# checking
final_df.head()

,Pokemon,Set,Number,Rarity,Condition,3/18 to 3/24,3/25 to 3/31,4/1 to 4/7,4/8 to 4/14,4/15 to 4/21,...,3/10 to 3/16,Popularity,Average,Min,Max,Range,Inverted Pop.,Weighted Price (Pop.),Price Fluctuations (Pop.),Price Deviation from Pop
0,Gengar,SWSH08: Fusion Strike,271/264,Secret Rare,Near Mint,228.68,252.39,289.66,312.91,314.11,...,679.85,2,422.19,228.68,744.80,516.12,360,151988.4,185803.2,274.07
1,Gengar,SV05: Temporal Forces,193/162,Ultra Rare,Near Mint,19.05,13.13,13.78,12.70,11.52,...,22.57,2,15.16,9.10,25.13,16.03,360,5457.6,5770.8,-132.96
2,Gengar,SWSH11: Lost Origin Trainer Gallery,TG06/TG30,Ultra Rare,Near Mint,2.47,2.57,2.98,2.35,2.60,...,16.61,2,7.01,2.35,19.98,17.63,360,2523.6,6346.8,-141.11
3,Charizard,Base Set,004/102,Holo Rare,Near Mint,385.66,386.97,370.05,369.79,360.99,...,462.27,1,383.34,323.85,462.27,138.42,361,138385.74,49969.62,176.03
4,Charizard,SM - Burning Shadows,150/147,Secret Rare,Near Mint,397.94,397.94,413.30,413.30,408.31,...,418.99,1,422.71,397.94,439.93,41.99,361,152598.31,15158.39,215.40
